<b>1. Import libraries</b>

In [1]:
from pytrends.request import TrendReq
import pandas as pd
import plotly
import plotly.express as px
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline 

from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

<b>2. Initialize</b>

In [2]:
pytrends = TrendReq()

<b>3. General settings</b>

In [3]:
keywords = ['python']
cat = '0'
geo = 'ID'
gprop = 'youtube'

timeframes = [
    'today 5-y',
    'today 3-y',
    'today 12-m',
    'today 5-m',
    'today 3-m', 
    'today 1-m',
    'now 1-d',
    'now 7-d',
    'now 1-H',
    'now 4-H'
]

In [4]:
pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=timeframes[4])

<b>4. Timeseries trends data</b>

In [5]:
# pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=timeframes[4])
timeseries = pytrends.interest_over_time()
timeseries.reset_index(inplace=True)
timeseries.rename(columns={'date':'datetime', keywords[0]: 'value', 'isPartial':'partial'}, inplace=True)
timeseries.head()

,datetime,value,partial
0,2021-12-01,39,False
1,2021-12-02,76,False
2,2021-12-03,48,False
3,2021-12-04,32,False
4,2021-12-05,53,False


In [6]:
tf = [
    'today 5-y',
    'today 3-y',
    'today 12-m',
    'today 5-m',
    'today 3-m', 
    'today 1-m',
    'now 1-d',
    'now 7-d',
    'now 1-H',
    'now 4-H'
]

In [12]:
from numpy.random import random

from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select
from bokeh.models import DatetimeTickFormatter

pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=tf[4])

x = timeseries.datetime
y = timeseries.value

def get_data():
    ts = pytrends.interest_over_time()
    ts.reset_index(inplace=True)
    ts.rename(columns={'date':'datetime', keywords[0]: 'value', 'isPartial':'partial'}, inplace=True)
    return dict(x=ts.datetime, y=ts.value)

def modify_doc(doc):
    source = ColumnDataSource(data=get_data())
    
    p = figure(toolbar_location=None)
    p = figure(
        title=f"YouTube Trends ({keywords[0].title()}) \n",
        width=800,
        height=350,
        x_axis_label="",
        y_axis_label=""
    )
    
    p.line(x='x', y='y', source=source, color="#8888cc", line_width=3, alpha=0.8)
    p.xaxis[0].formatter = DatetimeTickFormatter(days=["%b %d, %Y"])
 
    select = Select(title="Timeframe", value="today 3-m", options=tf)
    
    def update_data(attrname, old, new):
        pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=select.value)
        source.data = get_data()
        
    select.on_change('value', update_data)

    layout = column(row(select), row(p))

    doc.add_root(layout)

show(modify_doc)